## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import os
import matplotlib.pyplot as plt
import numpy as np

# Import the random module.
import random
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

import os,sys,inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir) 
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file.
city_data_df = pd.read_csv(parentdir+'\\Weather_Database\\WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kovrov,RU,56.36,41.32,54.59,73,100,7.61,overcast clouds
1,1,Ukiah,US,39.15,-123.21,84.20,26,1,6.93,clear sky
2,2,Porbandar,IN,21.64,69.61,80.60,88,20,4.70,haze
3,3,Vasterhaninge,SE,59.12,18.10,60.01,54,33,14.25,scattered clouds
4,4,Qaanaaq,GL,77.48,-69.36,23.23,81,0,8.34,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Ukiah,US,39.15,-123.21,84.20,26,1,6.93,clear sky
2,2,Porbandar,IN,21.64,69.61,80.60,88,20,4.70,haze
6,6,Kapaa,US,22.08,-159.32,82.40,65,90,10.29,overcast clouds
9,9,Avarua,CK,-21.21,-159.78,78.80,83,1,8.05,light rain
18,18,Guerrero Negro,MX,27.98,-114.06,74.32,74,0,14.99,clear sky
19,19,Acapulco,MX,16.86,-99.89,77.00,94,90,10.29,moderate rain
22,22,Rikitea,PF,-23.12,-134.97,71.71,84,97,11.12,overcast clouds
25,25,Akyab,MM,20.15,92.90,84.65,79,66,9.17,broken clouds
26,26,Faanui,PF,-16.48,-151.75,77.40,78,37,14.20,scattered clouds
29,29,Abu Zabad,SD,12.35,29.25,74.59,73,100,7.18,overcast clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
preferred_cities_df.count()


City_ID                315
City                   315
Country                315
Lat                    315
Lng                    315
Max Temp               315
Humidity               315
Cloudiness             315
Wind Speed             315
Current Description    315
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ukiah,US,84.20,clear sky,39.15,-123.21,
2,Porbandar,IN,80.60,haze,21.64,69.61,
6,Kapaa,US,82.40,overcast clouds,22.08,-159.32,
9,Avarua,CK,78.80,light rain,-21.21,-159.78,
18,Guerrero Negro,MX,74.32,clear sky,27.98,-114.06,
19,Acapulco,MX,77.00,moderate rain,16.86,-99.89,
22,Rikitea,PF,71.71,overcast clouds,-23.12,-134.97,
25,Akyab,MM,84.65,broken clouds,20.15,92.90,
26,Faanui,PF,77.40,scattered clouds,-16.48,-151.75,
29,Abu Zabad,SD,74.59,overcast clouds,12.35,29.25,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"


    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    

    # 6e. Make request and retrieve the JSON data from the search.
    hotels = requests.get(base_url, params=params).json() 
    
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
    

    
   
    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [10]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Ukiah,US,84.20,clear sky,39.15,-123.21,Hampton Inn Ukiah
2,Porbandar,IN,80.60,haze,21.64,69.61,Lords Inn Porbandar
6,Kapaa,US,82.40,overcast clouds,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
9,Avarua,CK,78.80,light rain,-21.21,-159.78,
18,Guerrero Negro,MX,74.32,clear sky,27.98,-114.06,
19,Acapulco,MX,77.00,moderate rain,16.86,-99.89,HS Hotsson Smart Acapulco
22,Rikitea,PF,71.71,overcast clouds,-23.12,-134.97,Pension Maro'i
25,Akyab,MM,84.65,broken clouds,20.15,92.90,
26,Faanui,PF,77.40,scattered clouds,-16.48,-151.75,Bora-Bora Pearl Beach Resort & Spa
29,Abu Zabad,SD,74.59,overcast clouds,12.35,29.25,


In [11]:
hotel_df["Hotel Name"].replace("",np.nan,inplace=True)

In [12]:
hotel_df[hotel_df["Hotel Name"] == "" ].count()

City                   0
Country                0
Max Temp               0
Current Description    0
Lat                    0
Lng                    0
Hotel Name             0
dtype: int64

In [13]:
hotel_df["Hotel Name"].isnull().sum()

305

In [14]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()

In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map.
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer) 

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))